In [46]:
import pandas as pd
import json
import statistics

df = pd.DataFrame({"year": range(2012, 2025)})

for i in range(1, 24):
    with open("./data/rent_data/" + str(i) + ".json", "r") as f:
        text = json.loads(f.read())

    rows = text["Trends"]["TrendChart"]["Rows"]
    start = rows[:4]
    values = [i[1] for i in start]
    year_mean = [round(statistics.mean(values), 2)]
    del rows[:4]
    for j in range(0, len(rows), 12):
        year = rows[j:j+12]
        values = [val[1] for val in year]
        year_mean.append(round(statistics.mean(values), 2))
    
    df[str(i) + ". District"] = year_mean

overall = []
for index, row in df.iloc[:, 1:24].iterrows():
    overall.append(round(statistics.mean(row), 2))

df.insert(1, "overall", overall)
df

,year,overall,1. District,2. District,3. District,4. District,5. District,6. District,7. District,8. District,...,14. District,15. District,16. District,17. District,18. District,19. District,20. District,21. District,22. District,23. District
0,2012,13.84,19.63,14.25,14.41,15.43,14.70,14.73,14.02,15.62,...,12.32,12.55,12.57,11.94,14.26,15.58,11.78,12.43,15.78,12.55
1,2013,14.04,19.41,14.62,15.02,15.19,13.99,15.31,14.82,15.07,...,12.58,12.05,12.91,12.74,14.93,15.83,12.29,12.63,16.05,12.41
2,2014,14.13,19.45,14.69,14.50,14.65,14.34,14.70,15.01,14.71,...,12.61,12.87,12.84,12.89,15.20,15.59,12.66,12.80,16.19,12.62
3,2015,14.18,19.23,14.92,14.43,15.18,14.24,14.90,14.98,14.63,...,12.53,12.66,13.27,12.82,14.92,15.49,12.95,13.11,16.03,13.10
4,2016,14.41,19.18,14.83,15.16,15.45,14.61,15.57,15.08,14.83,...,12.80,12.79,13.16,12.96,14.51,15.49,13.15,13.74,16.53,13.69
5,2017,14.80,19.45,15.37,15.81,15.80,14.91,16.02,15.53,15.01,...,14.38,13.24,13.42,13.54,14.97,15.73,13.39,13.83,16.31,13.89
6,2018,15.49,20.34,16.95,16.19,15.94,16.07,16.79,16.23,14.83,...,13.37,13.86,14.33,14.64,15.03,16.61,15.47,14.07,17.13,14.04
7,2019,15.37,19.84,16.18,15.89,16.14,15.32,16.21,15.80,15.25,...,13.96,13.96,14.53,14.46,15.47,16.66,14.87,14.92,16.09,14.41
8,2020,15.52,20.10,16.03,16.84,16.20,15.29,16.11,15.82,15.51,...,14.97,14.72,14.53,14.20,15.62,16.45,14.14,14.18,15.84,16.02
9,2021,15.79,20.22,15.51,17.04,16.15,15.32,16.61,15.86,15.67,...,14.75,15.09,15.24,14.17,15.52,16.46,13.71,15.43,16.52,15.67


In [47]:
df.to_csv("./data/average_rent_sqm.csv")

In [48]:
income_df = pd.read_csv("./data/vie-bez-biz-ecn-inc-sex-2002f.csv", sep=";", skiprows=1, usecols=[1,3,5,6,7])
rent_df = pd.read_csv("./data/average_rent_sqm.csv", index_col=0)

In [49]:
rent_df

,year,overall,1. District,2. District,3. District,4. District,5. District,6. District,7. District,8. District,...,14. District,15. District,16. District,17. District,18. District,19. District,20. District,21. District,22. District,23. District
0,2012,13.84,19.63,14.25,14.41,15.43,14.70,14.73,14.02,15.62,...,12.32,12.55,12.57,11.94,14.26,15.58,11.78,12.43,15.78,12.55
1,2013,14.04,19.41,14.62,15.02,15.19,13.99,15.31,14.82,15.07,...,12.58,12.05,12.91,12.74,14.93,15.83,12.29,12.63,16.05,12.41
2,2014,14.13,19.45,14.69,14.50,14.65,14.34,14.70,15.01,14.71,...,12.61,12.87,12.84,12.89,15.20,15.59,12.66,12.80,16.19,12.62
3,2015,14.18,19.23,14.92,14.43,15.18,14.24,14.90,14.98,14.63,...,12.53,12.66,13.27,12.82,14.92,15.49,12.95,13.11,16.03,13.10
4,2016,14.41,19.18,14.83,15.16,15.45,14.61,15.57,15.08,14.83,...,12.80,12.79,13.16,12.96,14.51,15.49,13.15,13.74,16.53,13.69
5,2017,14.80,19.45,15.37,15.81,15.80,14.91,16.02,15.53,15.01,...,14.38,13.24,13.42,13.54,14.97,15.73,13.39,13.83,16.31,13.89
6,2018,15.49,20.34,16.95,16.19,15.94,16.07,16.79,16.23,14.83,...,13.37,13.86,14.33,14.64,15.03,16.61,15.47,14.07,17.13,14.04
7,2019,15.37,19.84,16.18,15.89,16.14,15.32,16.21,15.80,15.25,...,13.96,13.96,14.53,14.46,15.47,16.66,14.87,14.92,16.09,14.41
8,2020,15.52,20.10,16.03,16.84,16.20,15.29,16.11,15.82,15.51,...,14.97,14.72,14.53,14.20,15.62,16.45,14.14,14.18,15.84,16.02
9,2021,15.79,20.22,15.51,17.04,16.15,15.32,16.61,15.86,15.67,...,14.75,15.09,15.24,14.17,15.52,16.46,13.71,15.43,16.52,15.67


In [26]:
income_df

,DISTRICT_CODE,REF_YEAR,INC_TOT_VALUE,INC_MAL_VALUE,INC_FEM_VALUE
0,90000,2002,18.217,20.709,15.424
1,90100,2002,25.463,31.961,18.536
2,90200,2002,16.439,18.301,14.282
3,90300,2002,18.701,21.444,15.804
4,90400,2002,20.325,23.641,16.876
...,...,...,...,...,...
475,91900,2021,29.903,34.540,25.235
476,92000,2021,21.201,22.241,19.916
477,92100,2021,24.381,26.587,21.926
478,92200,2021,26.722,29.850,23.454
